In [18]:
#以我們這組專題題目Avito中的title為例：

MAX_NUM_WORDS = 50000
MAX_SEQUENCE_LENGTH = 15
EMBEDDING_DIM = 300

#前面同之前的流程，建立 token 字典（此例字典大小為50000）, fit後轉換為數字list再截長補短（此例長度為50）
tokenizer = text.Tokenizer(num_words = MAX_NUM_WORDS)
tokenizer.fit_on_texts(all_text)
sequence = tokenizer.texts_to_sequences(df.title.str.lower())
pad_sequence=pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH) 

In [38]:
#先至fasttext官網下載所需語言的詞向量https://fasttext.cc/docs/en/crawl-vectors.html
#範例為俄語的cc.ru.300.vec向量檔案

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open('cc.ru.300.vec', encoding = 'utf8'))

#word_index ： 表示從語料庫之中保留多少個單詞。 因為Keras需要預留一個全零層， 所以+1
word_index = tokenizer.word_index
nb_words = min(MAX_NUM_WORDS, len(word_index)) + 1

#構建一個大小為[nb_words, EMBEDDING_DIM]的矩陣
#遍歷word_index。 將word在Fasttext模型之中對應vector複製過來
#排列順序按照Tokenizer在fit之後的詞順序，建立權重embedding_matrix之後留做稍後Embedding層使用
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS: continue
    embedding_vector = embeddings_index.get(word)
    
# words not found in embedding index will be all-zeros.
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
#keras 模型寫法：
#trainable = False，並將embedding權重指定為剛剛構建的embedding矩陣embedding_matrix。（weights=[embedding_matrix]）
seq_title_description = Input(shape=[15], name="seq_title_description")
emb_seq_title_description = Embedding(nb_words, EMBEDDING_DIM, weights = [embedding_matrix], trainable = False)(seq_title_description)


#其他程式寫法同一般keras模型
model = Model([seq_title_description], output)
model.compile(optimizer = 'adam',
                  loss= root_mean_squared_error,
                  metrics = [root_mean_squared_error])
